# Assignment ML

## Setup for assignment

In [12]:
from random import Random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [6]:
df = pd.read_csv(r"C:\Users\Vincent Lau\Google Drive\Quantitative finance\Machine learning\ml_assignment\data\train.csv",delimiter=";", decimal=",")


In [7]:
from cgi import test


train_df = df.sample(frac=0.6)

train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

test_df = df.drop(train_df.index)

test_df_x = test_df.iloc[:,1:]
test_df_y = test_df.iloc[:,0]

## Early Stopping

In [11]:
## Early stopping code
early_stopping_value = 0
early_stopping_accuracy = 0
for i in np.linspace(0.1,0.9,100):
    train_df = df.sample(frac=i)
    train_df_x = train_df.iloc[:,1:]
    train_df_y = train_df.iloc[:,0]

    test_df = df.drop(train_df.index)

    test_df_x = test_df.iloc[:,1:]
    test_df_y = test_df.iloc[:,0]
    
    clf = RandomForestClassifier(n_estimators=100, criterion="gini", max_depth=None)
    clf.fit(train_df_x, train_df_y)


    #Predict
    clf.predict(test_df_x)
    
    
    if (clf.score(test_df_x,test_df_y) - early_stopping_accuracy > 0.1):
        early_stopping_value = i
        early_stopping_accuracy = clf.score(test_df_x,test_df_y)
    
print("Best fraction to stop:", early_stopping_value)    

Best fraction to stop: 0.4797979797979798


## Tuning parameters

### Grid search


In [15]:
train_df = df.sample(frac=0.5)

train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

n_trees = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 100)]
max_features = [int(x) for x in np.linspace(start = 5, stop = 70, num = 5)]
criterion = ['gini', 'entropy']
max_depth = [int(x) for x in np.linspace(start = 3, stop = 100, num = 10)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 100, num = 10)]

random_grid = {'n_estimators': n_trees,
               'max_features': max_features,
               'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions= random_grid, n_iter = 100, cv=5)

rf_random.fit(train_df_x, train_df_y)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 13, 24, 35, 46, 56, 67,
                                                      78, 89, 100],
                                        'max_features': [5, 21, 37, 53, 70],
                                        'min_samples_split': [2, 12, 23, 34, 45,
                                                              56, 67, 78, 89,
                                                              100],
                                        'n_estimators': [100, 109, 118, 127,
                                                         136, 145, 154, 163,
                                                         172, 181, 190, 200,
                                                         209, 218, 227, 236,
                                                         245, 254, 263, 272,
               

In [16]:
# Print the randomized grid search parameters
# Results: n_estimators = 881, min_samples_split = 2, max_features = 70, max_depth = 78, criterion = entropy
print(rf_random.best_params_)

{'n_estimators': 881, 'min_samples_split': 2, 'max_features': 70, 'max_depth': 78, 'criterion': 'entropy'}


### Check for overfitting using the randomized grid search

In [24]:
rf2 = RandomForestClassifier(n_estimators=881, min_samples_split=2, max_features=70, max_depth=60, criterion='entropy')
rf3 = RandomForestClassifier(n_estimators=881, min_samples_split=2, max_features=70, max_depth=60, criterion='entropy')

train_df = df.sample(frac=.6)
train_df_x = train_df.iloc[:,1:]
train_df_y = train_df.iloc[:,0]

test_df = df.drop(train_df.index)

test_df_x = test_df.iloc[:,1:]
test_df_y = test_df.iloc[:,0]


rf2.fit(train_df_x, train_df_y)
rf3.fit(test_df_x, test_df_y)

#Predict
rf2.predict(test_df_x)
rf3.predict(train_df_x)
print("rf2:",rf2.score(test_df_x,test_df_y))
print("rf3:", rf3.score(train_df_x,train_df_y))


rf2: 0.9145942408376964
rf3: 0.8843552258346061
